# Install necessary packages

In [1]:
# !pip install --upgrade pip
# !pip install bs4
# !pip install requests
# !pip install html5lib
# !pip install lxml
# !pip install selenium==4.8.0
# !pip install webdriver-manager
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn


In [3]:
url = "https://www.turkishclass101.com/blog/2021/06/10/best-turkish-proverbs/#1"


In [4]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
 
# Define a custom user agent
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
 
# Set up Chrome options
options = uc.ChromeOptions()
options.add_argument("--headless")
options.add_argument(f"user-agent={my_user_agent}")
 
# Initialize Chrome WebDriver with the specified options
driver = uc.Chrome(options=options)
 
# Make a request to your target website.
driver.get(url=url)

# Close the driver
# driver.quit()


In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser') # driver.page_source is the html code of the page

In [6]:
idioms_table = soup.find_all('figure', class_='wp-block-table') # find all the tables in the page

In [7]:
# a single idiom table
# first tr is the turkish idiom. Second td of first tr is the idiom
# second tr is the literal translation. Second td of second tr is the literal translation
# third tr is the english equivalent. third tr has only one td. It contains explanation, example sentence and english equivalent sentence.
# fourth tr is the example

html_content = '''
<figure class="wp-block-table">
    <table border="1" cellspacing="0" width="100%">
        <tbody>
            <tr>
                <td rowspan="4">1</td>
                <td width="30%"><strong><span class="has-inline-color has-vivid-purple-color">Turkish</span></strong></td>
                <td><em>Zaman her şeyin ilacıdır.</em></td>
            </tr>
            <tr>
                <td><strong><span class="has-inline-color has-pale-cyan-blue-color">Literally</span></strong></td>
                <td>Time is the medicine of everything.</td>
            </tr>
            <tr>
                <td><strong><span class="has-inline-color has-vivid-cyan-blue-color">Equivalent in English</span></strong></td>
                <td>Time is the best medicine.</td>
            </tr>
            <tr>
                <td colspan="3">
                    As time passes, all the troubles we experience are forgotten or the sorrow we feel decreases.<br/><br/>
                    <u>Example:</u><br/><br/>
                    <em>Üzülme, bugünler de geçecek; zaman her şeyin ilacıdır.</em><br/><br/>
                    “Don’t worry, these days will be over, too; time is the best medicine.”
                </td>
            </tr>
        </tbody>
    </table>
</figure>
'''



In [23]:
import re
import pandas as pd

def extract_idioms(idioms_table):
    """
    Extracts information from a list of idiom tables.

    Parameters:
    idioms_table (list): A list of BeautifulSoup objects representing idiom tables.

    Returns:
    tuple: A tuple of lists containing the extracted information. The lists are in the following order:
        - idioms: A list of strings representing the Turkish idioms.
        - Literals: A list of strings representing the literal translations of the idioms.
        - equivalents: A list of strings representing the English equivalents of the idioms.
        - explanations: A list of strings representing the explanations of the idioms.
        - tr_sentences: A list of strings representing the example sentences in Turkish.
        - en_sentences: A list of strings representing the example sentences in English.
    """
    idioms = []
    Literals = []
    equivalents = []
    explanations = []
    tr_sentences = []
    en_sentences = []

    for idiom_table in idioms_table:
        for idx,tr in enumerate(idiom_table.find_all('tr')):
            if idx == 0:
                idiom = tr.find_all('td')[2].text
                idioms.append(idiom)
            elif idx == 1:
                literal = tr.find_all('td')[1].text
                Literals.append(literal)
            elif idx == 2:
                try:
                    equivalent = tr.find_all('td')[1].text
                    equivalents.append(equivalent)
                except:
                    equivalents.append(None)
            elif idx == 3:
                raw = str(tr.find_all('td')) # convert to string for regex
                print(raw)
                explanation = raw[raw.index('[<td colspan="3">')+len('[<td colspan="3">'):raw.index("<br/><br/>")] # extract explanation
                explanations.append(explanation)
                tr_sentence = raw[raw.index('<em>')+len('<em>'):raw.index('</em>')]
                tr_sentences.append(tr_sentence)    
                en_sentence = raw[raw.index('</em><br/><br/>')+len('</em><br/><br/>'):raw.index('</td>')]
                en_sentences.append(en_sentence)

    return (idioms, Literals, equivalents, explanations, tr_sentences, en_sentences)




In [25]:
import re
import pandas as pd

def extract_idioms(idioms_table):
    """
    Extracts information from a list of idiom tables.

    Parameters:
    idioms_table (list): A list of BeautifulSoup objects representing idiom tables.

    Returns:
    tuple: A tuple of lists containing the extracted information. The lists are in the following order:
        - idioms: A list of strings representing the Turkish idioms.
        - Literals: A list of strings representing the literal translations of the idioms.
        - equivalents: A list of strings representing the English equivalents of the idioms.
        - explanations: A list of strings representing the explanations of the idioms.
        - tr_sentences: A list of strings representing the example sentences in Turkish.
        - en_sentences: A list of strings representing the example sentences in English.
    """
    idioms = []
    Literals = []
    equivalents = []
    explanations = []
    tr_sentences = []
    en_sentences = []

    for idiom_table in idioms_table:
        for idx,tr in enumerate(idiom_table.find_all('tr')):
            if idx == 0:
                idiom = tr.find_all('td')[2].text
                idioms.append(idiom)
            elif idx == 1:
                literal = tr.find_all('td')[1].text
                Literals.append(literal)
            elif idx == 2:
                if "Equivalent in English" in tr.find_all('td')[0].text:
                        equivalent = tr.find_all('td')[1].text
                        equivalents.append(equivalent)
                else:
                    equivalents.append(None)
                    raw = str(tr.find_all('td')) # convert to string for regex
                    explanation = raw[raw.index('[<td colspan="3">')+len('[<td colspan="3">'):raw.index("<br/><br/>")] # extract explanation
                    explanations.append(explanation)
                    tr_sentence = raw[raw.index('<em>')+len('<em>'):raw.index('</em>')]
                    tr_sentences.append(tr_sentence)    
                    en_sentence = raw[raw.index('</em><br/><br/>')+len('</em><br/><br/>'):raw.index('</td>')]
                    en_sentences.append(en_sentence)
                    
                    continue
                    
            elif idx == 3:
                raw = str(tr.find_all('td')) # convert to string for regex
                print(raw)
                explanation = raw[raw.index('[<td colspan="3">')+len('[<td colspan="3">'):raw.index("<br/><br/>")] # extract explanation
                explanations.append(explanation)
                tr_sentence = raw[raw.index('<em>')+len('<em>'):raw.index('</em>')]
                tr_sentences.append(tr_sentence)    
                en_sentence = raw[raw.index('</em><br/><br/>')+len('</em><br/><br/>'):raw.index('</td>')]
                en_sentences.append(en_sentence)

    return (idioms, Literals, equivalents, explanations, tr_sentences, en_sentences)




In [28]:
len(idioms_table)

30

In [26]:
idioms, Literals, equivalents, explanations, tr_sentences, en_sentences = extract_idioms(idioms_table)

[<td colspan="3">As time passes, all the troubles we experience are forgotten or the sorrow we feel decreases.<br/><br/><u>Example:</u><br/><br/><em>Üzülme, bugünler de geçecek; zaman her şeyin ilacıdır.</em><br/><br/>“Don’t worry, these days will be over, too; time is the best medicine.”</td>]
[<td colspan="3">If you hold onto something you have for long enough, it will eventually become useful.<br/><br/><u>Example:</u><br/><br/><em>İyi ki kızımın bebek arabasını saklamışım, şimdi senin çok işine yarayacak. Eee, sakla samanı gelir zamanı.</em><br/><br/>“Fortunately, I saved my daughter’s stroller. It will be very useful for you now. See, keep a thing for seven years and you’ll find a use for it.”  </td>]
[<td colspan="3">This proverb emphasizes that time is a valuable resource.<br/><br/><u>Example:</u><br/><br/><em>Bir an önce işe gitmeliyim. Eee, ne de olsa vakit nakittir.</em><br/><br/>“I have to go to work as soon as possible. Well, after all, time is money.”  </td>]
[<td colspan="

In [27]:
len(idioms), len(Literals), len(equivalents), len(explanations), len(tr_sentences), len(en_sentences)

(30, 30, 30, 30, 30, 30)